Ở bài tập 1 – Homework 1 có các yêu cầu sau:
Dựa trên bộ dữ liệu tự thu thập từ các trang báo mạng Tiếng Việt (5 lớp/ mỗi lớp 10 mẩu
tin), hãy thực hiện các yêu cầu sau:
1. Tiền xử lý dữ liệu với Beautiful Soup, re,...
2. Tách từ (Tokenize) sử dụng thư viện pyvi hay underthesea
3. Trích xuất đặc trưng TF-IDF bằng thư viện sklearn
4. Đánh giá bộ dữ liệu với giải thuật KNN bằng phương pháp 5-Fold (k-fold)
5. Huấn luyện dữ liệu cho bài toán phân loại văn bản với tỷ lệ dữ liệu 8:2 (8 phần
train, 2 phần test) sử dụng đặc trưng TF-IDF và 2 giải thuật Bayes, SVM.
6. Tính độ đo F1 score
7. Tính độ đo Accuracy
8. Tính độ đo Confusion Matrix
9. So sánh kết quả các độ đo 6,7,8 với 2 giải thuật học máy ở trên
10.Lưu model với giải thuật đạt kết quả tốt nhất
11.Xây dựng ứng dụng phân loại văn bản với đầu vào là 1 văn bản bất kỳ có thể tự
ngõ hay từ 1 file, in kết quả ra màn hình

Hãy cải tiến bài 1 (nội dung 3-11) theo yêu cầu sau:


#1. Sử dụng đặc trưng TF-IDF với 2 mô hình học sâu RNN và GRU


In [4]:
!pip install requests beautifulsoup4
!pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 24.5 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
import pandas as pd
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from bs4 import BeautifulSoup
import requests
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, Dense

In [7]:
url_data = pd.read_excel('/content/drive/MyDrive/MCS/Deep Learning/Week 1/Homework1/Homework1_DATA.xlsx')
print(url_data)

                                                  url    category
0   https://vnexpress.net/dong-thap-se-nhan-chuyen...    Thời sự 
1   https://vnexpress.net/cao-toc-nha-trang-cam-la...    Thời sự 
2   https://vnexpress.net/moi-mon-cho-luong-huu-vi...    Thời sự 
3   https://vnexpress.net/lo-ngai-khi-nang-toc-do-...    Thời sự 
4   https://vnexpress.net/nang-thoi-han-thi-thuc-d...    Thời sự 
5   https://vnexpress.net/bao-tang-ky-uc-lang-da-m...    Thời sự 
6   https://vnexpress.net/oto-bep-dum-sau-cu-tong-...    Thời sự 
7   https://vnexpress.net/ong-nguyen-phu-cuong-tho...    Thời sự 
8   https://vnexpress.net/nguyen-bi-thu-tinh-uy-la...    Thời sự 
9   https://vnexpress.net/tong-bi-thu-khong-de-loi...    Thời sự 
10  https://vnexpress.net/vai-chin-som-giam-gia-mo...  Kinh doanh
11  https://vnexpress.net/loi-nhuan-nganh-phan-bon...  Kinh doanh
12  https://vnexpress.net/argentina-nang-lai-suat-...  Kinh doanh
13  https://vnexpress.net/bat-dong-san-nghi-duong-...  Kinh doanh
14  https:

In [8]:
X_raw = []
Y_raw = []
for _, row in url_data.iterrows():
    url = row['url']
    print(url)
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, "lxml")
    # data = soup.find("section", attrs={"class": "section page-detail top-detail"}).find_all('p', attrs={"class": "Normal"})
    data = soup.find_all('p', attrs={"class": "Normal"})
    content = ""
    for element in data:
      # print(element.get_text())
      content += element.get_text()
    print(content)
    sentences = content.split(".")
    print('sentences: ', sentences)
    #raw data
    X_raw = X_raw + sentences
    Y_raw = Y_raw + [row['category']]*len(sentences)
    # print('len X_raw: ', len(X_raw))
    # print("X_raw: ", X_raw)
    # print('len X_raw: ', len(X_raw))

    # print("Y_raw: ", Y_raw)
    # Y_raw.append(row['category'])
    # break
    #Tokenize
    # content_token = ViTokenizer.tokenize(content).split(' ')
    # print(len(content_token))


https://vnexpress.net/dong-thap-se-nhan-chuyen-giao-seu-tu-thai-lan-4605565.html
Ngày 16/5, Phó chủ tịch UBND Đồng Tháp Nguyễn Phước Thiện cho biết cơ quan chức năng hai nước tiếp tục trao đổi, thống nhất các bước để nhận sếu đầu đỏ ở vườn thú Nakhon Ratchasima (tỉnh Nakhon), cùng các giải pháp giúp sếu thích nghi điều kiện sống mới. Hiện, vườn quốc gia Tràm chim chuẩn bị xong khu vực chăm sóc sếu, cử cán bộ sang Thái Lan học tập kinh nghiệm.Sếu đầu đỏ thuộc động vật quý hiếm nên việc chuyển giao phải qua nhiều công đoạn. Bên tiếp nhận sếu có trách nhiệm bồi hoàn một khoản kinh phí tượng trưng cho công tác chăm sóc, thuần dưỡng (không được xem là mua bán). Ngoài ra, sếu được xem là "quốc bảo" của người Thái, do đó đại diện vườn thú của họ sẽ chọn hai động vật ở Việt Nam để tiến hành trao đổi.Theo ông Thiện, theo lộ trình cặp sếu đầu tiên sẽ được vườn thú ở Thái Lan chuyển giao cho Đồng Tháp năm 2024. Đây là hai sếu trưởng thành 6 tháng tuổi, đang trong giai đoạn trưởng thành. Địa phươn

In [29]:
print(len(X_raw), len(Y_raw))
print(X_raw)
print(Y_raw)

1384 1384
['Ngày 16/5, Phó chủ tịch UBND Đồng Tháp Nguyễn Phước Thiện cho biết cơ quan chức năng hai nước tiếp tục trao đổi, thống nhất các bước để nhận sếu đầu đỏ ở vườn thú Nakhon Ratchasima (tỉnh Nakhon), cùng các giải pháp giúp sếu thích nghi điều kiện sống mới', ' Hiện, vườn quốc gia Tràm chim chuẩn bị xong khu vực chăm sóc sếu, cử cán bộ sang Thái Lan học tập kinh nghiệm', 'Sếu đầu đỏ thuộc động vật quý hiếm nên việc chuyển giao phải qua nhiều công đoạn', ' Bên tiếp nhận sếu có trách nhiệm bồi hoàn một khoản kinh phí tượng trưng cho công tác chăm sóc, thuần dưỡng (không được xem là mua bán)', ' Ngoài ra, sếu được xem là "quốc bảo" của người Thái, do đó đại diện vườn thú của họ sẽ chọn hai động vật ở Việt Nam để tiến hành trao đổi', 'Theo ông Thiện, theo lộ trình cặp sếu đầu tiên sẽ được vườn thú ở Thái Lan chuyển giao cho Đồng Tháp năm 2024', ' Đây là hai sếu trưởng thành 6 tháng tuổi, đang trong giai đoạn trưởng thành', ' Địa phương kỳ vọng 4 năm tới tiếp nhận khoảng 20 sếu, vừa

In [30]:
#IF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X_raw)
Y = np.array(Y_raw)

#number category
le = preprocessing.LabelEncoder()
le.fit(Y)
Y = le.transform(Y)
print(X.shape)
print(Y)

(1384, 2505)
[4 4 4 ... 0 0 0]


In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [32]:
X_test.shape

(277, 2505)

In [33]:
y_train.shape

(1107,)

In [41]:
label_binarizer = LabelBinarizer()
y_train_one_hot = label_binarizer.fit_transform(y_train)
#Reshape dữ liệu thành định dạng phù hợp cho mạng RNN
X_train = X_train.toarray().reshape(-1, 1, 2505)
X_test = X_test.toarray().reshape(-1, 1, 2505)

# Xây dựng mạng RNN
model = Sequential()
model.add(SimpleRNN(units=32, input_shape=(1, 2505)))
model.add(Dense(units=5, activation='softmax'))

# Biên dịch mô hình
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Huấn luyện mạng RNN
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32)

# Đánh giá mô hình trên tập kiểm tra
loss, accuracy = model.evaluate(X_test, label_binarizer.transform(y_test))
print("Accuracy: {:.2f}%".format(accuracy * 100))

Epoch 1/10
35/35 [==============================] - 2s 5ms/step - loss: 1.5658 - accuracy: 0.4354
Epoch 2/10
35/35 [==============================] - 0s 4ms/step - loss: 1.4164 - accuracy: 0.7715
Epoch 3/10
35/35 [==============================] - 0s 4ms/step - loss: 1.2300 - accuracy: 0.8473
Epoch 4/10
35/35 [==============================] - 0s 5ms/step - loss: 1.0157 - accuracy: 0.8916
Epoch 5/10
35/35 [==============================] - 0s 4ms/step - loss: 0.8086 - accuracy: 0.9322
Epoch 6/10
35/35 [==============================] - 0s 4ms/step - loss: 0.6356 - accuracy: 0.9368
Epoch 7/10
35/35 [==============================] - 0s 4ms/step - loss: 0.5042 - accuracy: 0.9404
Epoch 8/10
35/35 [==============================] - 0s 4ms/step - loss: 0.4076 - accuracy: 0.9449
Epoch 9/10
35/35 [==============================] - 0s 5ms/step - loss: 0.3372 - accuracy: 0.9539
Epoch 10/10
9/9 [==============================] - 0s 3ms/step - loss: 0.4958 - accuracy: 0.8845
Accuracy: 88.45%


In [44]:
# Xây dựng mạng GRU
model = Sequential()
model.add(GRU(units=32, input_shape=(1, 2505)))
model.add(Dense(units=5, activation='softmax'))

# Biên dịch mô hình
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Huấn luyện mạng GRU
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32)

# Đánh giá mô hình trên tập kiểm tra
loss, accuracy = model.evaluate(X_test, label_binarizer.transform(y_test))
print("Accuracy: {:.2f}%".format(accuracy * 100))

Epoch 1/10
35/35 [==============================] - 4s 8ms/step - loss: 1.5836 - accuracy: 0.4173
Epoch 2/10
35/35 [==============================] - 0s 8ms/step - loss: 1.4871 - accuracy: 0.5592
Epoch 3/10
35/35 [==============================] - 0s 8ms/step - loss: 1.3448 - accuracy: 0.7660
Epoch 4/10
35/35 [==============================] - 0s 7ms/step - loss: 1.1492 - accuracy: 0.8618
Epoch 5/10
35/35 [==============================] - 0s 8ms/step - loss: 0.9238 - accuracy: 0.9061
Epoch 6/10
35/35 [==============================] - 0s 7ms/step - loss: 0.7121 - accuracy: 0.9350
Epoch 7/10
35/35 [==============================] - 0s 7ms/step - loss: 0.5471 - accuracy: 0.9413
Epoch 8/10
35/35 [==============================] - 0s 7ms/step - loss: 0.4283 - accuracy: 0.9494
Epoch 9/10
35/35 [==============================] - 0s 8ms/step - loss: 0.3450 - accuracy: 0.9494
Epoch 10/10
9/9 [==============================] - 1s 4ms/step - loss: 0.4899 - accuracy: 0.8736
Accuracy: 87.36%


# 2- Sử dụng Word embedding của Keras với 2 mô hình học sâu RNN và GRU

In [50]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Input data
sentences = X_raw
# Corresponding labels
le = preprocessing.LabelEncoder()
le.fit(Y)
Y = le.transform(Y)
labels = Y

# Tokenize sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(sentences)

# Pad sequences
max_len = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Convert labels to one-hot vectors
num_classes = len(np.unique(labels))
one_hot_labels = np.eye(num_classes)[labels]

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, one_hot_labels, test_size=0.2, random_state=42)

# Define the RNN model
embedding_dim = 100
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(SimpleRNN(units=32))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Predict the topics of new sentences
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))


Epoch 1/10
35/35 [==============================] - 3s 30ms/step - loss: 1.5958 - accuracy: 0.2611
Epoch 2/10
35/35 [==============================] - 1s 29ms/step - loss: 1.1807 - accuracy: 0.7588
Epoch 3/10
35/35 [==============================] - 1s 29ms/step - loss: 0.7038 - accuracy: 0.9061
Epoch 4/10
35/35 [==============================] - 1s 28ms/step - loss: 0.3677 - accuracy: 0.9494
Epoch 5/10
35/35 [==============================] - 1s 28ms/step - loss: 0.2257 - accuracy: 0.9575
Epoch 6/10
35/35 [==============================] - 1s 29ms/step - loss: 0.1663 - accuracy: 0.9603
Epoch 7/10
35/35 [==============================] - 1s 40ms/step - loss: 0.1320 - accuracy: 0.9639
Epoch 8/10
35/35 [==============================] - 2s 60ms/step - loss: 0.1149 - accuracy: 0.9702
Epoch 9/10
35/35 [==============================] - 2s 48ms/step - loss: 0.1079 - accuracy: 0.9711
Epoch 10/10
9/9 [==============================] - 0s 8ms/step - loss: 1.7537 - accuracy: 0.4332
Accuracy: 43

In [51]:
# Define the RNN model
embedding_dim = 100
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GRU(units=32))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Predict the topics of new sentences
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Epoch 1/10
35/35 [==============================] - 8s 87ms/step - loss: 1.5918 - accuracy: 0.2855
Epoch 2/10
35/35 [==============================] - 2s 58ms/step - loss: 1.4555 - accuracy: 0.4607
Epoch 3/10
35/35 [==============================] - 2s 55ms/step - loss: 1.0115 - accuracy: 0.7299
Epoch 4/10
35/35 [==============================] - 2s 55ms/step - loss: 0.7007 - accuracy: 0.8681
Epoch 5/10
35/35 [==============================] - 2s 54ms/step - loss: 0.4457 - accuracy: 0.9268
Epoch 6/10
35/35 [==============================] - 2s 55ms/step - loss: 0.2686 - accuracy: 0.9494
Epoch 7/10
35/35 [==============================] - 3s 78ms/step - loss: 0.1666 - accuracy: 0.9657
Epoch 8/10
35/35 [==============================] - 3s 84ms/step - loss: 0.1195 - accuracy: 0.9711
Epoch 9/10
35/35 [==============================] - 2s 54ms/step - loss: 0.0981 - accuracy: 0.9747
Epoch 10/10
9/9 [==============================] - 1s 16ms/step - loss: 0.6345 - accuracy: 0.8014
Accuracy: 8